In [2]:
import pandas as pd 
from pandas_datareader import data, wb

# Gather and clean the data : 

In [3]:
# The stocks we'll use for this analysis
# Stock_list = ['GE','BAC','JDSU','XOM','^GSPC'] ( Note that the stock JDSU is not working)

Stock_list = ['GE','BAC','XOM']

# Set up End and Start times for data grab

startDate = '2000-01-01'
endDate = '2013-05-31'

# #For loop for grabing yahoo finance data and setting as a dataframe
# panel_data = data.DataReader('INPX', 'yahoo', start, end)

SNP = data.DataReader('^GSPC','yahoo',startDate,endDate)
for stock in Stock_list:   
    # Set DataFrame as the Stock Ticker
    globals()[stock] = data.DataReader(stock,'yahoo',startDate,endDate)


In [4]:
# Merge the close price for all the stocks:
stocksClose =pd.concat([GE['Adj Close'].rename('GE'),BAC['Adj Close'].rename('BAC'),XOM['Adj Close'].rename('XOM'),SNP['Adj Close'].rename('S&P')], axis=1)

In [ ]:
# 1.2 Federal Reserve Economic Data (FRED) from the St. Louis Federal Reserve
#       
#
#             Returns historical data for any symbol at the website
#               http://research.stlouisfed.org/fred2/
#
# Series name | Description
# 
# DGS3MO      | 3-Month Treasury, constant maturity rate
# DGS1        | 1-Year Treasury, constant maturity rate
# DGS5        | 5-Year Treasury, constant maturity rate
# DGS10       | 10-Year Treasury, constant maturity rate
#
# DAAA        | Moody's Seasoned Aaa Corporate Bond Yield 
# DBAA        | Moody's Seasoned Baa Corporate Bond Yield 
#
# DCOILWTICO  | Crude Oil Prices: West Text Intermediate (WTI) - Cushing, Oklahoma

In [5]:
fredList = ['DGS3MO','DGS1','DGS5','DGS10','DAAA','DBAA','DCOILWTICO']

In [6]:
fredData = data.DataReader(fredList,'fred',startDate,endDate)

In [15]:
# The join= 'inner' options takes the index intersection of all data frames. The join ='Outer' ( default) takes the union.
allData =pd.concat([fredData,stocksClose],axis=1,join='inner').rename_axis('Date')


In [16]:
allData.to_csv('rawData.csv')

In [17]:
############################################################################################################################################################################# DATA CLEANING:
###########################################################################################################################################################################

# Count the number of NaN values: 
allData.isna().sum()

DGS3MO        24
DGS1          24
DGS5          24
DGS10         24
DAAA          24
DBAA          24
DCOILWTICO    14
GE             0
BAC            0
XOM            0
S&P            0
dtype: int64

In [18]:
# We want to fill the missing value with the previews non NanN value, i.e forward fill. 
cleanData = allData.fillna(method='ffill')

In [88]:
# The only missing values is the first entry in DGS3MO
cleanData.isna().sum()

DGS3MO        0
DGS1          0
DGS5          0
DGS10         0
DAAA          0
DBAA          0
DCOILWTICO    1
GE            0
BAC           0
XOM           0
S&P           0
dtype: int64

In [20]:
# Save the clean data: 
cleanData.to_csv('cleanData.csv')